# Closure Notes

These are notes based on Real Python's [article](https://realpython.com/python-closure/) with motivation to better understand Python [decorators](Decorators.ipynb).

A [closure](https://en.wikipedia.org/wiki/Closure_(computer_programming)) also known as a function closure, is a technique to store a function with an environment. The function has local variables mapped to the enclosing scope.

In Python, a closure is a nested function where the inner function is returned by the outer function, along with parameter bindings defined in the outer function.

## Background

### Inner Functions

This is simply a nested function. Just like in the format of many Elements of Programming Interview Python solutions, nesting functions can control scope:

In [1]:
def outer_func():
    name = 'Jonathan'
    def inner_func():
        print(f'Hello, {name}!')
    inner_func()

outer_func()

Hello, Jonathan!


### Function Closures

The defining feature of a function closure separating them from other inner functions is that it **returns the inner function**. The anatomy of a function closure is the following:
- outer function defining the scope
- variables local to the outer function
- inner function defined inside the outer function

In [2]:
def outer_func():
    name = 'Jonathan'
    def inner_func():
        print(f'Hello, {name}')
    return inner_func

outer_func()

<function __main__.outer_func.<locals>.inner_func()>

In [4]:
greeter = outer_func()
greeter()

Hello, Jonathan


Lambda functions can be used to build closures:

In [5]:
def outer_func():
    name = 'Jonathan'
    return lambda: print(f'Hello, {name}!')

greeter = outer_func()
greeter()

Hello, Jonathan!


### Captured Variables

In this example, the inner function has access to variables that are defined even after itself, demonstrating the subtleties of scope.

In [7]:
def outer_func(outer_arg):
    local_var = 'Outer local variable'
    def closure():
        print(outer_arg)
        print(local_var)
        print(another_local_var)
    another_local_var = 'Another local variable'
    return closure

closure = outer_func('Outer argument')
closure()

Outer argument
Outer local variable
Another local variable


#### Updating Immutable Objects

In [14]:
def make_counter():
    count = 0
    def counter():
        nonlocal count # look for a predefined variable instead of a new one
        count += 1
        return count
    return counter

counter = make_counter()
counter()

1

In [15]:
counter()

2

In [16]:
counter()

3

#### Updating Mutable Objects

In [18]:
def make_appender():
    items = []
    def appender(new_item):
        items.append(new_item)
        return items
    return appender

appender = make_appender()
appender('First item')

['First item']

In [19]:
appender('Second item')

['First item', 'Second item']

In [20]:
appender('Third item')

['First item', 'Second item', 'Third item']

## Creating Closures to Retain State

### Creating Factory Functions

In [21]:
def make_root_calculator(root_degree, precision=2):
    def root_calculator(number):
        return round(pow(number, 1 / root_degree), precision)
    return root_calculator

square_root = make_root_calculator(2, 4)

square_root(42)

6.4807

In [22]:
cubic_root = make_root_calculator(3)
cubic_root(42)

3.48

### Building Stateful Function

In [25]:
def cumulative_average():
    data = []
    def average(value):
        data.append(value)
        return sum(data) / len(data)
    return average

# more efficient implementation
def cumulative_average():
    total = 0
    n = 0
    def average(value):
        nonlocal total
        nonlocal n
        n += 1
        total += value
        return total / n
    return average
        


stream_average = cumulative_average()

stream_average(12)

12.0

In [26]:
stream_average(13)

12.5

In [27]:
stream_average(11)

12.0

In [28]:
stream_average(10)

11.5

### Providing Callback Functions

The `callback()` function returns a closure object that is passed onto the `command` argument, which takes callable objects without arguments. This is a workaround to pass parameters.

In [29]:
import tkinter as tk

app = tk.Tk()
app.title('GUI App')
app.geometry('320x240')

label = tk.Label(
    app,
    font=('Helvetica', 16, 'bold'),
)
label.pack()

def callback(text):
    def closure():
        label.config(text=text)
        
    return closure

button = tk.Button(
    app,
    text='Greet',
    command=callback('Hello, World!'),
)

button.pack()

app.mainloop()
    

## Writing Decorators with Closures

There are two types of decorators in Python:
- function based
- class based

Here is a simple function based implementation with and without the decorator, and using the syntax:

In [34]:
def greet():
    print('Hello, World!')

greet()

Hello, World!


In [35]:
def decorator(function):
    def closure():
        print('Do something before calling the function.')
        function()
        print('Do something after calling the function.')
    return closure

def greet():
    print('Hello, World!')

greet = decorator(greet)
greet()

Do something before calling the function.
Hello, World!
Do something after calling the function.


In [36]:
@decorator
def greet():
    print('Hello, World!')

greet()

Do something before calling the function.
Hello, World!
Do something after calling the function.


## Impelementing Memoization with Closures

In [37]:
def memoize(function):
    cache = {}
    def closure(number):
        if number not in cache:
            cache[number] = function(number)
        return cache[number]
    return closure

In [39]:
from time import sleep
from timeit import timeit

def slow_operation(number):
    sleep(0.5)

timeit("[slow_operation(number) for number in [2, 3, 4, 2, 3, 4]]",
       globals=globals(),
       number=1,
      )

3.0021948000066914

In [40]:
@memoize
def slow_operation(number):
    sleep(0.5)

timeit("[slow_operation(number) for number in [2, 3, 4, 2, 3, 4]]",
       globals=globals(),
       number=1,
      )

1.501583400007803

## Achieving Encapsulation with Closures

#### Private Methods by Naming Convention

In [41]:
class Stack:
    def __init__(self):
        self._items = []

    def push(self, item):
        self._items.append(item)

    def pop(self):
        return self._items.pop()

stack = Stack()
stack.push(1)
stack.push(2)
stack.push(3)

stack.pop()

3

In [42]:
stack._items

[1, 2]

#### Private Methods by Closures

In [43]:
def Stack():
    _items = []

    def push(item):
        _items.append(item)

    def pop():
        return _items.pop()

    def closure():
        pass

    closure.push = push
    closure.pop = pop
    return closure

stack = Stack()
stack.push(1)
stack.push(2)
stack.push(3)

stack.pop()

3

In [44]:
stack._items

AttributeError: 'function' object has no attribute '_items'

In [45]:
stack

<function __main__.Stack.<locals>.closure()>

## Exploring Alternatives to Closures

Instead of closures, which can be difficult to reason about, we can use classes with the `__call__()` method:

In [46]:
def make_root_calculator(root_degree, precision=2):
    def root_calculator(number):
        return round(pow(number, 1 / root_degree), precision)
    return root_calculator


square_root = make_root_calculator(2, 4)
square_root(42)

6.4807

In [47]:
class RootCalculator:
    def __init__(self, root_degree, precision=2):
        self.root_degree = root_degree
        self.precision = precision

    def __call__(self, number):
        return round(pow(number, 1 / self.root_degree), self.precision)

square_root = RootCalculator(2, 4)
square_root(42)

6.4807

In [48]:
cubic_root = RootCalculator(3)
cubic_root(42)

3.48